In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
# Veri setini yükle
dataset = load_dataset("nbertagnolli/counsel-chat")

Repo card metadata block was not found. Setting CardData to empty.


In [11]:
dataset.keys()

dict_keys(['train'])

In [12]:
# Eksik verileri temizle
def clean_data(dataset):
    # Pandas DataFrame'e dönüştür
    df = pd.DataFrame(dataset)

    # Eksik değerleri doldur
    df = df.dropna(subset=["questionText", "answerText"])  # Boş sorular ve cevaplar çıkarılır

    # Veriyi sadece gerekli kolonlarla sınırlı tut
    df = df[["questionText", "answerText"]]

    return df

# Veri setini temizle
train_df = clean_data(dataset["train"])

# Eğitim ve doğrulama setine ayır
train_texts, val_texts = train_test_split(
    train_df, test_size=0.1, random_state=42  # %10 doğrulama seti
)

# Eğitim ve doğrulama setlerini kontrol et
print(f"Eğitim seti boyutu: {len(train_texts)}")
print(f"Doğrulama seti boyutu: {len(val_texts)}")

Eğitim seti boyutu: 2350
Doğrulama seti boyutu: 262


In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import torch

# DialoGPT model ve tokenizer'ı yükle
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_name)

In [15]:
# Pad token'ı ayarla
tokenizer.pad_token = tokenizer.eos_token

# Veriyi token haline getirme
def tokenize_data(data):
    inputs = tokenizer(
        data["questionText"].tolist(),
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    targets = tokenizer(
        data["answerText"].tolist(),
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

# Eğitim ve doğrulama verilerini tokenleştir
train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)

In [16]:
# PyTorch Dataset sınıfı
class DialogueDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: tensor[idx] for key, tensor in self.encodings.items()}

# Dataset'leri oluştur
train_dataset = DialogueDataset(train_encodings)
val_dataset = DialogueDataset(val_encodings)

In [17]:
# Eğitim parametreleri
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=500,
    save_strategy="epoch",
    report_to="none",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
# Trainer nesnesini oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Eğitimi başlat
trainer.train()

<ipython-input-18-d84339e03ea3>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,5.883747
2,6.085400,5.819700
3,6.085400,5.810968


TrainOutput(global_step=882, training_loss=6.003578514739229, metrics={'train_runtime': 888.2407, 'train_samples_per_second': 7.937, 'train_steps_per_second': 0.993, 'total_flos': 1636834974105600.0, 'train_loss': 6.003578514739229, 'epoch': 3.0})

In [19]:
# Modeli kaydet
model.save_pretrained("./counsel_chat_dialoGPT")
tokenizer.save_pretrained("./counsel_chat_dialoGPT")

print("Model eğitildi ve kaydedildi.")

Model eğitildi ve kaydedildi.


In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Eğitilen modeli ve tokenizer'ı yükle
model_path = "./counsel_chat_dialoGPT"  # Modelin kaydedildiği yol
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Chatbot fonksiyonu
def chat_with_bot():
    print("Chatbot'a hoş geldiniz! Çıkmak için 'exit' yazabilirsiniz.")
    chat_history = ""

    max_history_length = 3  # Sohbet geçmişini sınırlamak için kullanılacak maksimum mesaj sayısı

    while True:
        # Kullanıcıdan giriş al
        user_input = input("Siz: ")

        # Çıkış komutunu kontrol et
        if user_input.lower() == "exit":
            print("Chatbot oturumu sonlandırıldı.")
            break

        # Sohbet geçmişini güncelle ve sınırla
        chat_history += f"User: {user_input}\nBot: "
        chat_history_list = chat_history.split(tokenizer.eos_token)
        if len(chat_history_list) > max_history_length:
            chat_history_list = chat_history_list[-max_history_length:]
        chat_history = tokenizer.eos_token.join(chat_history_list)

        # Girişi tokenleştir ve modelden yanıt al
        inputs = tokenizer.encode(chat_history + tokenizer.eos_token, return_tensors="pt", truncation=True)

        try:
            outputs = model.generate(
                inputs,
                max_length=150,
                temperature=1.2,  # Daha rastgele yanıtlar için sıcaklık artırıldı
                top_p=0.95,  # Top-p sampling
                top_k=50,    # Top-k sampling
                do_sample=True,  # Sampling etkinleştirildi
                repetition_penalty=1.2,  # Tekrarlı ifadeleri azaltır
                pad_token_id=tokenizer.eos_token_id,  # Padding token ayarlandı
            )

            # Yanıtı çözümle
            response = tokenizer.decode(outputs[:, inputs.shape[-1]:][0], skip_special_tokens=True)

            # Yanıt boşsa veya anlamlı değilse
            if not response.strip():
                response = "Bunu anlamadım, tekrar dener misiniz?"

            print(f"Chatbot: {response}")

            # Sohbet geçmişine bot yanıtını ekle
            chat_history += response + tokenizer.eos_token

        except Exception as e:
            print("Bir hata oluştu:", str(e))
            print("Lütfen girişinizi kontrol edin ve tekrar deneyin.")

# Chatbot'u başlat
chat_with_bot()


Chatbot'a hoş geldiniz! Çıkmak için 'exit' yazabilirsiniz.
Siz: hello
Chatbot:  the
Siz: i feel bad
Chatbot: Bunu anlamadım, tekrar dener misiniz?
Siz: how are you
Chatbot: ! do your a


KeyboardInterrupt: Interrupted by user